In [ ]:
!pip install accelerate peft transformers datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 22.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EarlyStoppingCallback

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", device_map="auto", torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset, Dataset

ds = load_dataset("eshangj/stackoverflow_q_and_a_sample")

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/23.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15451 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(ds['train'])
df = df[df['answer_vote'] > 0]
df = df.drop(['answer_vote', 'question_id', 'creation_date', 'question_vote', 'link'], axis=1)

# Tokenize the prompts and answers separately
tokenized_inputs = df.apply(lambda x: tokenizer(x['question'], truncation=True, padding="max_length"), axis=1)
tokenized_labels = df.apply(lambda x: tokenizer(x['accepted_answer'], truncation=True, padding="max_length"), axis=1)

df['input_ids'] = [x['input_ids'] for x in tokenized_inputs.values]
df['attention_mask'] = [x['attention_mask'] for x in tokenized_inputs.values]
df['labels'] = [x['input_ids'] for x in tokenized_labels.values]

df = df.drop(['question', 'accepted_answer'], axis=1)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

peft_model_args = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
)

peft_model = get_peft_model(model, peft_model_args)
peft_model = peft_model.to(device)

In [ ]:
train_df = df.sample(frac=0.8, random_state=42)
eval_df = df.drop(train_df.index)

In [ ]:
peft_model.train()  # Set model to training mode
for param in peft_model.parameters():
    if param.requires_grad:
        param.requires_grad_(True)

In [ ]:
# Convert DataFrame to Dataset (assuming df exists from previous steps)
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./stackoverflow_flan_finetune",
    per_device_train_batch_size=4,
    learning_rate=5e-4,
    num_train_epochs=1,
    logging_steps=100,
    save_strategy="steps",
    report_to="tensorboard",
    fp16=False,
    gradient_checkpointing=False,
    eval_strategy="steps",
    load_best_model_at_end=True,
)

# Instantiate Trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

/tmp/ipython-input-829545955.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', '__index_level_0__'],
    num_rows: 11995
})

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
100,7.734300,2.363281
200,2.109000,1.661133
300,2.297200,5.343750


TrainOutput(global_step=300, training_loss=4.0468131510416665, metrics={'train_runtime': 678.3413, 'train_samples_per_second': 17.683, 'train_steps_per_second': 4.421, 'total_flos': 824970313728000.0, 'train_loss': 4.0468131510416665, 'epoch': 0.10003334444814939})

In [ ]:
peft_model.save_pretrained("./stackoverflow_flan_finetune")